Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import BatchNormalization
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier

Load Dataset

In [2]:
# load dataset
df = np.loadtxt("data_oversampled.csv", skiprows = 1, delimiter=",")
# split into input (X) and output (Y) variables
x = df[:,0:4]
y = df[:,4]

Standardisation

In [3]:
#Train, Test and Split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state=101)
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

Tuning the dropout regularization rate and weight constraints

In [25]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 5 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(128, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.796382 using {'model__dropout_rate': 0.4, 'model__weight_constraint': 1.0}
0.785755 (0.020275) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.785755 (0.020275) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.785755 (0.020275) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.785755 (0.020275) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.785755 (0.020275) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.787256 (0.012694) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.787256 (0.012694) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.787256 (0.012694) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.787256 (0.012694) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.787256 (0.012694) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.778165 (0.034873) with: {'model__dr

In [26]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 4 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=0)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.799433 using {'model__dropout_rate': 0.5, 'model__weight_constraint': 1.0}
0.797904 (0.015057) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.797904 (0.015057) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.797904 (0.015057) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.797904 (0.015057) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.797904 (0.015057) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.770555 (0.022029) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.770555 (0.022029) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.770555 (0.022029) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.770555 (0.022029) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.770555 (0.022029) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.788785 (0.015038) with: {'model__dr

In [21]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 3 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(32, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.802518 using {'model__dropout_rate': 0.3, 'model__weight_constraint': 1.0}
0.788799 (0.021335) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.788799 (0.021335) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.788799 (0.021335) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.788799 (0.021335) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.788799 (0.021335) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.790307 (0.020373) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.790307 (0.020373) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.790307 (0.020373) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.790307 (0.020373) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.790307 (0.020373) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.782718 (0.021993) with: {'model__dr

In [27]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 2 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(16, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=0)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.794894 using {'model__dropout_rate': 0.3, 'model__weight_constraint': 1.0}
0.787298 (0.031631) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.770576 (0.033145) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.770576 (0.033145) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.770576 (0.033145) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.770576 (0.033145) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.790335 (0.033090) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.788827 (0.037020) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.788827 (0.037020) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.788827 (0.037020) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.788827 (0.037020) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.791850 (0.029429) with: {'model__dr

In [29]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 1 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(8, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.779715 using {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.778200 (0.042662) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.770617 (0.053515) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.770617 (0.053515) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.770617 (0.053515) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.770617 (0.053515) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.776664 (0.035042) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.779715 (0.042374) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.779715 (0.042374) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.779715 (0.042374) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.779715 (0.042374) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.772125 (0.046162) with: {'model__dr

In [24]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# NO hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(4, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.775149 using {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.769081 (0.048155) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.769060 (0.037061) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.769060 (0.037061) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.769060 (0.037061) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.769060 (0.037061) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.767538 (0.039907) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.775149 (0.040577) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.775149 (0.040577) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.775149 (0.040577) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.775149 (0.040577) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.759935 (0.033357) with: {'model__dr

Combined Grid Search of Learning rate, momentum and epochs

In [4]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 2 hidden layers (256-256-8)
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    # define the keras model
    model = Sequential()

    model.add(Dense(256, input_dim=4, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
epochs = [50, 100, 150, 200, 250, 300]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum, epochs=epochs)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=16, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.802511 using {'epochs': 250, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.2}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.4}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.6}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.9}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.0}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.2}
0.778207 (0.043094) with: {'epochs': 50, 'optimizer__learning_rate': 0

In [5]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 2 hidden layers (256-164-8)
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    # define the keras model
    model = Sequential()

    model.add(Dense(256, input_dim=4, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(164, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
epochs = [50, 100, 150, 200, 250, 300]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum, epochs=epochs)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=16, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.799474 using {'epochs': 300, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.2}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.4}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.6}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.9}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.0}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.2}
0.784260 (0.030711) with: {'epochs': 50, 'optimizer__learning_rate': 0

In [6]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 2 hidden layers (512-512-8)
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    # define the keras model
    model = Sequential()

    model.add(Dense(512, input_dim=4, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
epochs = [50, 100, 150, 200, 250, 300]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum, epochs=epochs)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=16, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.800927 using {'epochs': 300, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.2}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.4}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.6}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.9}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.0}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.2}
0.778200 (0.040317) with: {'epochs': 50, 'optimizer__learning_rate': 0

In [7]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 2 hidden layers (512-334-8)
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    # define the keras model
    model = Sequential()

    model.add(Dense(512, input_dim=4, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(334, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_constraint=MaxNorm(1.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
epochs = [50, 100, 150, 200, 250, 300]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum, epochs=epochs)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=16, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.800941 using {'epochs': 300, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.2}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.4}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.6}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.9}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.0}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.2}
0.782766 (0.042454) with: {'epochs': 50, 'optimizer__learning_rate': 0